In [9]:
search_for = 78
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004313468933105469
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 116059691
type: [1, 1, 1, 1, 78] 78
cases of this type: 37015056
10000 18.42962465699177
20000 28.304696732355893
30000 70.39992516542756
40000 56.0719090908386
50000 59.53535805259085
60000 62.39280486334329
70000 58.78064896601934
80000 57.96432988170666
90000 56.584314824401226
100000 58.892057210369
110000 57.28376527340913
120000 62.73690620713758
130000 64.50226827051162
140000 58.29269278493725
150000 58.8731780682159
160000 57.92561628046994
170000 55.62832104463616
180000 56.482572755236816
190000 59.19424387916691
200000 58.61763390649933
210000 54.203463868860844
220000 58.17869373734339
230000 41.05332253378527
240000 10.093947348985738
250000 10.154522210978898
260000 35.57731189307757
270000 62.36912507773044
280000 52.3948095116104
290000 55.48949728888327
300000 59.32806229148435
310000 55.35985088876912
320000 

3090000 9.110150995722082
3100000 9.07998005310342
3110000 39.594866789973835
3120000 42.29969262910338
3130000 44.82801596490868
3140000 43.11503491773361
3150000 45.370251119038286
3160000 44.344367448130214
3170000 47.169107463785224
3180000 44.76376239463417
3190000 46.48618080963146
3200000 47.02895750556824
3210000 46.118637652254094
3220000 42.48388328752497
3230000 44.76951537561937
3240000 46.10792784675772
3250000 46.09975279714048
3260000 44.75436806288338
3270000 47.393834151345274
3280000 31.317405494691187
3290000 21.55571392343249
3300000 23.006431128538015
3310000 42.911502166625766
3320000 47.425975777115326
3330000 54.656995460651686
3340000 49.092359143042216
3350000 50.227781848826055
3360000 27.09967198631602
3370000 18.52101712767948
3380000 18.274907184200483
3390000 42.481985636110906
3400000 51.043751156795174
3410000 46.52787218165378
3420000 48.88823855393348
3430000 49.074836655938796
3440000 46.91184347509188
3450000 46.92630617082159
3460000 43.31666938902

6210000 44.89494552095669
6220000 44.922510199050166
6230000 39.92752009504742
6240000 34.762109288946206
6250000 19.125080886582857
6260000 14.673085821546218
6270000 27.602004540018203
6280000 44.352643822262515
6290000 40.39280772573044
6300000 42.75900261995587
6310000 42.037213068149974
6320000 37.48078174164838
6330000 41.70817472836903
6340000 42.855213425137926
6350000 33.96419369251631
6360000 40.580615576541994
6370000 37.23465397003655
6380000 37.4592577249156
6390000 35.861657984579146
6400000 20.803190054538373
6410000 8.665475344804644
6420000 9.081080050496618
6430000 29.22105914005167
6440000 38.591073511960424
6450000 39.17852660198533
6460000 40.38878913309516
6470000 35.09558917603518
6480000 34.76369615067443
6490000 41.46348241224125
6500000 44.06098703760725
6510000 41.01578808553931
6520000 39.53482501414037
6530000 39.74371345321488
6540000 37.139729080335655
6550000 39.1386859716629
6560000 26.732803038395463
6570000 19.315855118364574
6580000 21.28021047141707

9340000 39.360879275369946
9350000 35.6059936449791
9360000 39.985651070877196
9370000 25.169838910229743
9380000 17.355764541635992
9390000 19.566995352011023
9400000 35.262678193651986
9410000 38.80448415540246
9420000 42.73195005298232
9430000 39.91051136957769
9440000 42.30732198644773
9450000 40.08918995828369
9460000 40.54508802155183
9470000 40.648412875507056
9480000 43.14667665044858
9490000 46.710149857922396
9500000 53.32602620562454
9510000 46.67628232770727
9520000 43.991602042296165
9530000 40.671762126473745
9540000 37.003763574505676
9550000 42.865678509674964
9560000 43.91744469801056
9570000 41.916105540328616
9580000 42.02390980122762
9590000 40.99590124914781
9600000 43.61857120802373
9610000 16.83783830795255
9620000 14.35932129227843
9630000 14.755733749084637
9640000 39.0154867740087
9650000 39.81517652191621
9660000 31.869288101234492
9670000 42.70743986034675
9680000 39.72869693380593
9690000 40.26582452126389
9700000 39.00694493420698
9710000 40.35433495978199

12370000 41.826600600738786
12380000 39.03094911972231
12390000 37.90203070378636
12400000 40.39390670710442
12410000 38.28560499753408
12420000 39.84426601220513
12430000 39.49528945806045
12440000 34.501977785659314
12450000 40.67778566950109
12460000 35.67915881367068
12470000 36.985172197948756
12480000 38.11227033774309
12490000 22.96652216354202
12500000 16.21489060850461
12510000 14.27623615934478
12520000 26.735808193626912
12530000 35.12607973010945
12540000 40.16755407155573
12550000 32.16752107031974
12560000 36.1091764175963
12570000 19.89921927980596
12580000 8.22281543397959
12590000 7.782776572199979
12600000 27.491195680676576
12610000 35.627910032883086
12620000 36.909129326296274
12630000 38.20964430211066
12640000 36.46789791010058
12650000 25.091482056243876
12660000 17.707120277805142
12670000 11.899522082367447
12680000 32.1124996762193
12690000 31.927780471738412
12700000 36.490053266430486
12710000 34.755914730617825
12720000 35.67891432033385
12730000 35.676926

...

104470000 1.3163134498991371
104480000 2.491917067568163
104490000 1.492668381344835
104500000 2.161403644235975
104510000 2.4863348270171284
104520000 3.1527527380431826
104530000 2.3789631566131577
104540000 2.205937761711498
104550000 1.7980659456771544
104560000 1.173925860438115
104570000 2.192536935123066
104580000 1.1169112081412012
104590000 0.9636495445169277
104600000 1.5303226718899015
104610000 1.3293421863632666
104620000 2.8549180585311387
104630000 2.4523737894258897
104640000 2.8050794237552816
104650000 2.793013144355178
104660000 2.116828315124194
104670000 2.1770976801834503
104680000 1.9199521522872844
104690000 2.00452700135833
104700000 1.0833969917868709
104710000 1.911056259870569
104720000 1.8835177787128095
104730000 0.9718740776558851
104740000 2.3842990470582177
104750000 1.7861762381878297
104760000 2.3916434419797725
104770000 2.583517038606677
104780000 2.2703008798924684
104790000 1.0449563348674178
104800000 2.0413939581771956
104810000 2.49917214002358

107300000 0.7568557412039638
107310000 0.43339342835138894
107320000 0.510455734975954
107330000 0.48329447768698136
107340000 0.8017550810067522
107350000 0.6590348375433088
107360000 0.6785808165436387
107370000 0.826494631991148
107380000 0.6789502329422102
107390000 0.8230626266300082
107400000 1.102297319481492
107410000 0.7513752973370619
107420000 0.6924547111696402
107430000 0.7050611900878416
107440000 0.7566189400688741
107450000 0.8073231472116907
107460000 0.723844680496898
107470000 1.1362700275880295
107480000 0.866820148780048
best so far: 0
type: [1, 2, 1, 13, 3] 78
cases of this type: 355914
107490000 1.0610600942283606
107500000 1.0677022778606877
107510000 0.6749525897613962
107520000 0.6388453048212196
107530000 1.0062934863655766
107540000 0.7482018601205747
107550000 0.5438414532203012
107560000 0.6132506673671074
107570000 0.9613952979581951
107580000 1.2118986584235096
107590000 0.6961888894584642
107600000 0.8600986352768143
107610000 0.7406945525845621
1076200

109930000 1.0493462489670782
109940000 0.6962753108679851
109950000 0.5209886516031093
109960000 0.4548203892949024
109970000 1.0265707216283282
109980000 0.9537539479194216
109990000 1.0187797321657075
110000000 0.6559839567863344
110010000 0.5312175559018122
110020000 0.7892118895741701
110030000 0.7938628345089952
110040000 0.7268560609686838
110050000 0.6512672651898066
110060000 0.2838676403861642
110070000 0.35204171503641873
110080000 0.5245416433612241
110090000 0.8073500160226821
110100000 0.9170469786755575
110110000 0.390307124549124
110120000 0.4074806947127183
110130000 0.6276222199871738
110140000 0.6694271103057199
110150000 1.1093816018240255
110160000 0.5939275579382711
110170000 0.47952411775109505
110180000 0.754399194017291
110190000 1.0185273372724322
110200000 0.4517799958183633
110210000 0.4503599720239044
110220000 0.45111862721035223
110230000 0.43884182702594987
110240000 0.314140018137614
110250000 0.2548644803744224
110260000 0.3444833364453117
110270000 0.3

112130000 0.5029589940452576
112140000 0.6012938538609941
112150000 0.7688113017620577
112160000 0.587290221642673
112170000 0.5503538833134902
112180000 0.6851180982683234
112190000 0.4814336652422746
112200000 0.741655256909728
112210000 0.8931015670206678
112220000 0.8663889665240844
112230000 0.7759234822521807
112240000 0.5002131270578172
112250000 0.2405976116091013
112260000 0.46246540852158596
112270000 0.8092333902440734
112280000 0.9478423211354017
112290000 0.7256996026394897
112300000 0.512518518301169
112310000 0.2863582676754196
112320000 0.8423868178322315
112330000 0.8814215192685062
112340000 0.7954344199540615
112350000 0.47999736553108696
112360000 0.36769806541758776
112370000 0.2371988973795772
112380000 0.6016291045723888
112390000 0.5025173356766833
112400000 0.27683779737758635
112410000 0.15096217200732232
112420000 0.1708604815819396
112430000 0.15712045016276835
112440000 0.22927423039538994
112450000 0.22297910710363253
112460000 0.17637653280280033
11247000

114820000 0.05293985404841767
114830000 0.06201279570792119
114840000 0.07494874933529562
114850000 0.06168583581754234
114860000 0.051174715845108035
114870000 0.04996689083020554
114880000 0.0497714235544867
114890000 0.11714239513685304
114900000 0.1375153795374036
114910000 0.10863838939440913
114920000 0.07742129627090692
114930000 0.07637736733515395
114940000 0.07658031532606151
114950000 0.0814766308323741
114960000 0.08343578911796543
best so far: 0
type: [2, 1, 3, 13, 1] 78
cases of this type: 19773
114970000 0.055667920441051316
114980000 0.040285399483501905
best so far: 0
type: [2, 1, 13, 1, 3] 78
cases of this type: 13689
114990000 0.044156899111727876
best so far: 0
type: [2, 1, 13, 3, 1] 78
cases of this type: 4563
115000000 0.04305406226314439
best so far: 0
type: [2, 1, 39, 1, 1] 78
cases of this type: 1521
best so far: 0
type: [2, 3, 1, 1, 13] 78
cases of this type: 85683
115010000 0.06669898075952133
115020000 0.06487209468009074
115030000 0.1012388311379883
1150400